In [1]:
%pip install bitshuffle
%pip install Pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
#from ultralytics import YOLO
import time
import os
import random
import psutil
import scipy.ndimage
from PIL import Image
from PIL import ImageDraw
from pathlib import Path
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
%matplotlib inline

In [3]:
# import cv2

# # Load the image
# image = cv2.imread('/home/jliang/gbt-rfi/example.png', cv2.IMREAD_UNCHANGED)

# # Print shape and number of channels
# print(f"Shape: {image.shape}")

# if len(image.shape) == 2:
#     print("Image is grayscale (1 channel).")
# elif len(image.shape) == 3:
#     print(f"Image has {image.shape[2]} channels.")
# else:
#     print("Unexpected image format.")


In [4]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
36553,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36554,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36555,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36556,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [5]:
high_path = df['.h5 path'].iloc[-1]

In [6]:
df['.h5 path'] = df['.h5 path'].str.replace('0000.h5', '0002.h5', regex=False)
df = df.drop_duplicates(subset='.h5 path', keep='first').reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
30309,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30310,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30311,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30312,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [7]:
df = df[~df['.h5 path'].str.contains('spliced')].reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
1,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
2,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
3,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
4,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
...,...,...,...,...,...,...,...,...
29341,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29342,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29343,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29344,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [8]:
#path = df['.h5 path'].iloc[-1]

In [9]:
# fb_high = bl.Waterfall(high_path)
# high_data = fb_high.data # shape: (279; 1; 65,536)
# plt.imshow(10*np.log10(high_data).squeeze(), aspect='auto')
# #plt.xlim(1250, 1251)
# plt.show()

In [10]:
# fb_mid = bl.Waterfall(path)
# mid_data = fb_mid.data # shape: (279; 1; 65,536)
# plt.imshow(10*np.log10(mid_data).squeeze(), aspect='auto')
# plt.xlim(10000, 10003)
# plt.show()

In [11]:
# fb_mid = bl.Waterfall(path)
# mid_data = fb_mid.data # shape: (279; 1; 65,536)
# plt.imshow(10*np.log10(mid_data).squeeze(), aspect='auto')
# plt.xlim(1250, 1251)
# plt.show()

In [ ]:
df = df.drop(index = 17546)
df = df.sample(n=2000, random_state=42).reset_index(drop=True)

In [13]:
# def process_scan_for_yolo(index, h5_path, base_dir,
#                           coarse_channel_width=128,
#                           factor=7,
#                           dilation_size=(10,300),
#                           class_id=0):
#     """
#     For each coarse channel:
#       - generate a PNG of the waterfall slice,
#       - produce a .txt file of YOLO bboxes for each connected hit region.
#     """
#     out_dir = Path(out_dir)
#     img_dir = out_dir / "images"
#     img_dir.mkdir(parents=True, exist_ok=True)

#     stem = Path(h5_path).stem

#     # 1) load & log-scale
#     fb   = bl.Waterfall(h5_path, load_data=True)
#     data = 10 * np.log10(fb.data.squeeze())   # shape (ntime, nfreq)
#     nt, nf = data.shape
#     n_coarse = nf // coarse_channel_width

#     # 2) global threshold
#     m, s = np.median(data), np.std(data)
#     binary = data > (m + factor * s)

#     # 3) dilate to merge neighboring hits
#     binary = scipy.ndimage.maximum_filter(binary, size=dilation_size)

#     # 4) make directories
#     img_tr = Path(base_dir)/"train"/"images"
#     lbl_tr = Path(base_dir)/"train"/"labels"
#     img_vl = Path(base_dir)/"val"/"images"
#     lbl_vl = Path(base_dir)/"val"/"labels"
#     for d in (img_tr, lbl_tr, img_vl, lbl_vl):
#         d.mkdir(parents=True, exist_ok=True)

#     stem = Path(h5_path).stem

#     # 4) per-block: find components, save image + YOLO .txt
#     for i in range(n_coarse):
#         f0, f1 = i * coarse_channel_width, (i + 1) * coarse_channel_width
#         block = data[:, f0:f1]
#         mask  = binary[:, f0:f1]

#         # connected components
#         labeled, n_objs = scipy.ndimage.label(mask)

#         # 1) extract the slice
#         block = data[:, f0:f1]   # shape: (ntime, coarse_channel_width)

#         # 2) threshold & find connected components as before
#         mask  = binary[:, f0:f1]
#         labeled, n_objs = scipy.ndimage.label(mask)

#         # 3) convert block to 8-bit RGB JPEG
#         #    normalize just this block so it fills 0–255
#         arr8 = (255 * (block - block.min()) / block.ptp()).astype(np.uint8)
#         rgb  = np.stack([arr8]*3, axis=-1)        # (H, W, 3)
#         img = Image.fromarray(rgb)
#         img_name = f'img_{index:05d}.png'
#         out_path = os.path.join(img_dir, img_name)
#         img.save(out_path)


#     # 4) write the matching .txt with the same basename
#     txt_name = img_name.replace(".png", ".txt")
#     with open(img_dir / txt_name, "w") as ftxt:
#         for lab in range(1, n_objs + 1):
#             ys, xs = np.where(labeled == lab)
#             y_min, y_max = ys.min(), ys.max()
#             x_min, x_max = xs.min(), xs.max()
#             # normalized YOLO coords
#             x_c = ((x_min + x_max) / 2) / coarse_channel_width
#             y_c = ((y_min + y_max) / 2) / nt
#             w   = (x_max - x_min) / coarse_channel_width
#             h   = (y_max - y_min) / nt
#             ftxt.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")


#         # write YOLO .txt (one line per blob)
#         txt_fn = img_dir / f"{stem}_chan{i:03d}.txt"
#         with open(txt_fn, "w") as ftxt:
#             for lab in range(1, n_objs + 1):
#                 ys, xs = np.where(labeled == lab)
#                 y_min, y_max = ys.min(), ys.max()
#                 x_min, x_max = xs.min(), xs.max()

#                 # normalized YOLO coords
#                 x_c = ((x_min + x_max) / 2) / coarse_channel_width
#                 y_c = ((y_min + y_max) / 2) / nt
#                 w   = (x_max - x_min) / coarse_channel_width
#                 h   = (y_max - y_min) / nt

#                 ftxt.write(f"0 {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")


In [ ]:
fb = bl.Waterfall(df['.h5 path'].iloc[-1], load_data=True)
fb.info()


--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :               10501.46484375 MHz
            foff :         -0.00286102294921875 MHz
           ibeam :                               -1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                            65536
            nifs :                                1
     rawdatafile : guppi_59411_54386_HIP30264_0094.0000.raw
     source_name :                         HIP30264
         src_dej :                     -8:26:53.228
         src_raj :                      6:21:58.451
    telescope_id :                                6
           tsamp :                1.073741823999999
   tstart (ISOT) :          2021-07-16T15:06:26.000
    tstart (MJD) :                59

In [ ]:
# 4) SET UP LOGGING
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

def build_tasks(df, coarse_channel_width):
    """
    Build a flat list of (h5_path, channel_idx) from your DataFrame.
    Uses df['nchans'] if present, otherwise reads header to find nfreq.
    """
    tasks = []
    for _, row in df.iterrows():
        h5 = row[".h5 path"]
        fb = bl.Waterfall(h5, load_data=False)
        nfreq = fb.header.get("nchans")
        n_coarse = nfreq // coarse_channel_width

        for ch in range(n_coarse):
            tasks.append((h5, ch))
    return tasks

def split_tasks(tasks, train_frac=0.8, seed=42):
    """
    Shuffle & split the flat task list into train vs. val sets.
    Returns two sets of (h5_path, channel_idx).
    """
    random.seed(seed)
    shuffled = tasks.copy()
    random.shuffle(shuffled)
    cut = int(train_frac * len(shuffled))
    train = set(shuffled[:cut])
    val   = set(shuffled[cut:])
    return train, val

def process_file(job):
    """
    job is a tuple:
      (h5_path, [ch_idx,...], [global_idx,...],
       base_dir, coarse_channel_width, factor, dilation_size,
       class_id, train_set, val_set)
    """
    (h5_path,
     channels,
     global_indices,
     base_dir,
     cw,
     factor,
     dilation,
     class_id,
     train_set,
     val_set,
     pad_width) = job

    # 1) load the waterfall once
    fb   = bl.Waterfall(h5_path, load_data=True)
    data = 10 * np.log10(fb.data.squeeze())   # shape (ntime, nfreq)
    nt, nf = data.shape

    # 2) threshold + dilation
    M, S = np.median(data), np.std(data)
    binary = data > (M + factor * S)
    binary = scipy.ndimage.maximum_filter(binary, size=dilation)

    # 3) process each channel
    for ch_idx, gidx in zip(channels, global_indices):
        f0, f1 = ch_idx*cw, (ch_idx+1)*cw
        block  = data[:, f0:f1]
        mask   = binary[:, f0:f1]
        edge_bins = 5
        mask[:, :edge_bins] = False
        mask[:, -edge_bins:] = False
        labeled, n_objs = scipy.ndimage.label(mask)

        # decide train vs val
        subset = "train" if (h5_path, ch_idx) in train_set else "val"

        # make sure all dirs exist
        img_dir = Path(base_dir)/subset/"images"
        lbl_dir = Path(base_dir)/subset/"labels"
        vis_dir = Path(base_dir)/"visualization"/subset
        wf_dir  = Path(base_dir)/"waterfall"/subset
        for d in (img_dir, lbl_dir, vis_dir, wf_dir):
            d.mkdir(parents=True, exist_ok=True)

        # 3a) save the channel PNG
        arr8 = (255*(block - block.min())/block.ptp()).astype(np.uint8)
        rgb  = np.stack([arr8]*3, axis=-1)
        img  = Image.fromarray(rgb)
        fn = f"img_{gidx:0{pad_width}d}.png"
        img_path = img_dir/fn
        img.save(img_path)

        # 3b) save the YOLO .txt
        txt_path = lbl_dir/fn.replace(".png", ".txt")
        with open(txt_path, "w") as ftxt:
            for lab in range(1, n_objs + 1):
                ys, xs = np.where(labeled == lab)

                # pixel‐perfect extents (inclusive)
                ymin, ymax = ys.min(), ys.max()
                xmin, xmax = xs.min(), xs.max()

                # width/height in pixels
                w_pix = (xmax - xmin + 1)
                h_pix = (ymax - ymin + 1)

                # center in pixel coords
                x_center_pix = xmin + w_pix / 2.0
                y_center_pix = ymin + h_pix / 2.0

                # normalize to [0,1] by true image dims
                x_c = x_center_pix / float(cw)
                y_c = y_center_pix / float(nt)
                w_n = w_pix       / float(cw)
                h_n = h_pix       / float(nt)

                ftxt.write(f"{class_id} "
                        f"{x_c:.6f} {y_c:.6f} "
                        f"{w_n:.6f} {h_n:.6f}\n")

        logging.info(f"[{subset}] {fn}  (ch {ch_idx} of {Path(h5_path).name})")

        # —————————————————————————————
        # 4) Draw boxes on the saved PNG
        # —————————————————————————————
        vis_img = Image.open(img_path).convert("RGB")
        draw    = ImageDraw.Draw(vis_img)
        w_img, h_img = vis_img.size

        with open(txt_path) as ftxt:
            for line in ftxt:
                cls, x_c, y_c, bw, bh = map(float, line.split())
                xc = x_c * w_img
                yc = y_c * h_img
                bw_pix = bw * w_img
                bh_pix = bh * h_img
                x0 = int(xc - bw_pix/2)
                y0 = int(yc - bh_pix/2)
                x1 = int(xc + bw_pix/2)
                y1 = int(yc + bh_pix/2)
                draw.rectangle([x0, y0, x1, y1], outline="red", width=2)

        vis_img.save(vis_dir/fn)

        # —————————————————————————————
        # 5) Plot and save the full waterfall 
        # —————————————————————————————
        wf_name = f"{Path(fn).stem}_wf.png"
        wf_path = wf_dir/wf_name

        plt.imsave(
            wf_path,
            data,
            origin="lower",
            cmap="viridis"
        )
        logging.info(f"[{subset}] {wf_name}  (ch {ch_idx} of {Path(h5_path).name})")
        
if __name__ == "__main__":
    # === your settings ===
    BASE_DIR             = "/datax/scratch/jliang/channel_masking_dataset"
    COARSE_CHANNEL_WIDTH = 512          # for BL MR files
    FACTOR               = 7
    DILATION_SIZE        = (3, 10)
    CLASS_ID             = 0
    TRAIN_FRAC           = 0.8
    SEED                 = 42
    NUM_WORKERS          = 8

    # --- assume you already have a DataFrame `df` with at least 'h5_path' and optionally 'nchans' ---
    # df = pd.read_csv(...)  # or however you built it

    # 1) Build & split tasks
    tasks = build_tasks(df, COARSE_CHANNEL_WIDTH)
    train_set, val_set = split_tasks(tasks, TRAIN_FRAC, SEED)

    # 2) compute zero-pad width from total images
    pad_width = len(str(len(tasks) - 1))

    # 3) regroup tasks by file to load each .h5 only once
    jobs = {}
    for gidx, (h5, ch) in enumerate(tasks):
        jobs.setdefault(h5, {"chs": [], "gidxs": []})
        jobs[h5]["chs"].append(ch)
        jobs[h5]["gidxs"].append(gidx)

    # 4) prepare job‐tuples
    job_list = []
    for h5, info in jobs.items():
        job_list.append((
            h5,
            info["chs"],
            info["gidxs"],
            BASE_DIR,
            COARSE_CHANNEL_WIDTH,
            FACTOR,
            DILATION_SIZE,
            CLASS_ID,
            train_set,
            val_set,
            pad_width
        ))

    # 5) run in parallel, one process per file
    with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
        for _ in as_completed(exe.map(process_file, job_list)):
            pass

    logging.info("All done.")


In [ ]:
# #------------------------------------------------------------------------------
# # 1) The per-channel worker
# #------------------------------------------------------------------------------
# from concurrent.futures import ProcessPoolExecutor, as_completed

# def process_channel(h5_path, ch_idx, global_idx,
#                     base_dir,
#                     coarse_channel_width,
#                     factor,
#                     dilation_size,
#                     class_id):
#     """
#     Process one coarse channel of one H5
#     """
#     # --- load & log-scale ---
#     fb = bl.Waterfall(h5_path, load_data=True)
#     data = 10 * np.log10(fb.data.squeeze())  # (ntime, nfreq)
#     nt, nf = data.shape

#     # --- threshold & dilate ---
#     median, std = np.median(data), np.std(data)
#     binary = data > (median + factor * std)
#     binary = scipy.ndimage.maximum_filter(binary, size=dilation_size)

#     # --- extract this channel ---
#     f0, f1 = ch_idx*coarse_channel_width, (ch_idx+1)*coarse_channel_width
#     block = data[:, f0:f1]
#     mask = binary[:, f0:f1]

#     # --- connected components to get bboxes ---
#     labeled, n_objs = scipy.ndimage.label(mask)

#     # --- pick train vs val by global_idx or ch_idx if you prefer ---
#     # (example: 80% train by global index)
#     n_total = int((nf//coarse_channel_width))
#     # you can precompute a train_set instead of this if you like
#     split_point = int(0.8 * n_total)
#     is_train = (global_idx % n_total) < split_point  # or any rule you choose

#     subset = "train" if is_train else "val"
#     img_dir = Path(base_dir)/subset/"images"
#     lbl_dir = Path(base_dir)/subset/"labels"
#     img_dir.mkdir(parents=True, exist_ok=True)
#     lbl_dir.mkdir(parents=True, exist_ok=True)

#     # --- save the PNG ---
#     arr8 = (255*(block - block.min())/block.ptp()).astype(np.uint8)
#     rgb = np.stack([arr8]*3, axis=-1)
#     img = Image.fromarray(rgb)
#     img_name = f"img_{global_idx:04d}.png"
#     img.save(img_dir/img_name)

#     # --- save the YOLO .txt ---
#     with open(lbl_dir/img_name.replace(".png", ".txt"), "w") as f:
#         for lab in range(1, n_objs+1):
#             ys, xs = np.where(labeled == lab)
#             y0, y1 = ys.min(), ys.max()
#             x0, x1 = xs.min(), xs.max()
#             x_c = ((x0 + x1)/2) / coarse_channel_width
#             y_c = ((y0 + y1)/2) / nt
#             w = (x1 - x0) / coarse_channel_width
#             h = (y1 - y0) / nt
#             f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")

#     return global_idx


# #------------------------------------------------------------------------------
# # 2) Build the flat task list
# #------------------------------------------------------------------------------
# def build_tasks(df, coarse_channel_width):
#     """
#     From your DataFrame df with column 'h5_path',
#     returns a list of (h5_path, channel_idx, global_idx).
#     """
#     tasks = []
#     idx_counter = 0

#     for h5 in df['.h5 path']:
#         fb = bl.Waterfall(h5, load_data=False)
#         nchans = fb.header['data']['nchans']
#         n_coarse = nchans // coarse_channel_width

#         for ch in range(n_coarse):
#             tasks.append((h5, ch, idx_counter))
#             idx_counter += 1

#     return tasks


# #------------------------------------------------------------------------------#
# # 3) Kick off the parallel loop
# #------------------------------------------------------------------------------#
# def main(df,
#          base_dir,
#          num_workers=8,
#          coarse_channel_width=1024,
#          factor=7,
#          dilation_size=(10,300),
#          class_id=0):
#     # build tasks with global indices
#     tasks = build_tasks(df, coarse_channel_width)

#     # dispatch
#     with ProcessPoolExecutor(max_workers=num_workers) as exe:
#         futures = [
#             exe.submit(
#                 process_channel,
#                 h5, ch, idx,
#                 base_dir,
#                 coarse_channel_width,
#                 factor,
#                 dilation_size,
#                 class_id
#             )
#             for h5, ch, idx in tasks
#         ]

#         # progress
#         for fut in as_completed(futures):
#             idx = fut.result()
#             print(f"Done img_{idx:04d}.png")


# #------------------------------------------------------------------------------
# # 4) Run it
# #------------------------------------------------------------------------------
# if __name__ == "__main__":
#     # assume your notebook already has `df` loaded
#     BASE_DIR = "/datax/scratch/jliang/channel_masking_dataset"
#     main(df,
#          base_dir=BASE_DIR,
#          num_workers=8,
#          coarse_channel_width=1024,
#          factor=7,
#          dilation_size=(10,300),
#          class_id=0)


In [ ]:
# class ChannelMaskingDataset:
#     def __init__(self,
#                  base_dir,
#                  coarse_channel_width=128,
#                  factor=5,
#                  dilation_size=(10,300),
#                  class_id=0,
#                  train_frac=0.8,
#                  seed=42):
#         """
#         base_dir/
#           train/images, train/labels
#           val/images,   val/labels
#           visualization/train, visualization/val
#           waterfall/train,     waterfall/val
#         """
#         self.base_dir = Path(base_dir)
#         self.cc_width = coarse_channel_width
#         self.factor = factor
#         self.dilation_size = dilation_size
#         self.class_id = class_id
#         self.train_frac = train_frac
#         self.seed = seed

#         # set up all dirs
#         self.train_image_dir = self.base_dir/"train"/"images"
#         self.train_label_dir = self.base_dir/"train"/"labels"
#         self.val_image_dir = self.base_dir/"val"/"images"
#         self.val_label_dir = self.base_dir/"val"/"labels"
#         self.vis_train_dir = self.base_dir/"visualization"/"train"
#         self.vis_val_dir = self.base_dir/"visualization"/"val"
#         self.wf_train_dir = self.base_dir/"waterfall"/"train"
#         self.wf_val_dir = self.base_dir/"waterfall"/"val"

#         for d in (self.train_image_dir, self.train_label_dir,
#                   self.val_image_dir, self.val_label_dir,
#                   self.vis_train_dir, self.vis_val_dir,
#                   self.wf_train_dir, self.wf_val_dir):
#             d.mkdir(parents=True, exist_ok=True)

#     def process_scan(self, h5_path, visualize=False):
#         fb = bl.Waterfall(h5_path, load_data=True)
#         data = 10 * np.log10(fb.data.squeeze())      # (ntime, nfreq)
#         nt, nf = data.shape
#         n_coarse = nf // self.cc_width

#         # threshold + dilation
#         M, S = np.median(data), np.std(data)
#         binary = data > (M + self.factor * S)
#         binary = scipy.ndimage.maximum_filter(binary,
#                                              size=self.dilation_size)

#         # train/val split
#         idxs = list(range(n_coarse))
#         random.seed(self.seed)
#         random.shuffle(idxs)
#         cut = int(self.train_frac * n_coarse)
#         train_set = set(idxs[:cut])
#         val_set = set(idxs[cut:])

#         stem = Path(h5_path).stem

#         for i in range(n_coarse):
#             # slice out channel
#             f0, f1 = i*self.cc_width, (i+1)*self.cc_width
#             block  = data[:, f0:f1]
#             mask   = binary[:, f0:f1]

#             # CC → YOLO bboxes
#             labeled, n_objs = scipy.ndimage.label(mask)

#             # pick train vs val
#             if i in train_set:
#                 img_dir = self.train_image_dir
#                 lbl_dir = self.train_label_dir
#             else:
#                 img_dir = self.val_image_dir
#                 lbl_dir = self.val_label_dir

#             # --- save the channel PNG ---
#             arr8 = (255*(block - block.min())/block.ptp()).astype(np.uint8)
#             rgb  = np.stack([arr8]*3, axis=-1)
#             img  = Image.fromarray(rgb)

#             img_name = f"{stem}_chan{i:05d}.png"
#             img_path = img_dir / img_name
#             img.save(img_path)  # PNG by extension

#             # --- save the YOLO .txt ---
#             txt_path = lbl_dir / img_name.replace(".png", ".txt")
#             with open(txt_path, "w") as ftxt:
#                 for lab in range(1, n_objs+1):
#                     ys, xs = np.where(labeled == lab)
#                     y0, y1 = ys.min(), ys.max()
#                     x0, x1 = xs.min(), xs.max()

#                     # normalized coords
#                     x_c = ((x0 + x1)/2) / self.cc_width
#                     y_c = ((y0 + y1)/2) / nt
#                     w   = (x1 - x0)        / self.cc_width
#                     h   = (y1 - y0)        / nt

#                     ftxt.write(f"{self.class_id} "
#                                f"{x_c:.6f} {y_c:.6f} "
#                                f"{w:.6f} {h:.6f}\n")

#             if visualize:
#                 self._visualize_channel(i, img_path, txt_path, fb, train_set)

#     def _visualize_channel(self, i, img_path, txt_path, fb, train_set):
#         # pick visualization dirs
#         if i in train_set:
#             vis_dir = self.vis_train_dir
#             wf_dir  = self.wf_train_dir
#         else:
#             vis_dir = self.vis_val_dir
#             wf_dir  = self.wf_val_dir

#         # 1) draw boxes onto the saved PNG
#         vis_dir.mkdir(parents=True, exist_ok=True)
#         img = Image.open(img_path).convert("RGB")
#         draw = ImageDraw.Draw(img)
#         w, h = img.size

#         with open(txt_path) as f:
#             for line in f:
#                 cls, x_c, y_c, bw, bh = map(float, line.split())
#                 xc = x_c * w
#                 yc = y_c * h
#                 bw *= w
#                 bh *= h
#                 x0 = int(xc - bw/2)
#                 y0 = int(yc - bh/2)
#                 x1 = int(xc + bw/2)
#                 y1 = int(yc + bh/2)
#                 draw.rectangle([x0, y0, x1, y1], outline="red", width=2)

#         out_vis = vis_dir / img_path.name
#         img.save(out_vis)

#         # 2) plot full waterfall
#         wf_dir.mkdir(parents=True, exist_ok=True)
#         w_data = 10*np.log10(fb.data.squeeze())  # full (ntime, nfreq)
#         fig, ax = plt.subplots(figsize=(6.4,6.4), dpi=100)
#         ax.imshow(w_data, aspect="auto", origin="lower", cmap="viridis")
#         ax.axis("off")
#         plt.subplots_adjust(left=0,right=1,top=1,bottom=0)
#         wf_path = wf_dir / f"{img_path.stem}_wf.png"
#         fig.savefig(wf_path, bbox_inches="tight", pad_inches=0)
#         plt.close(fig)


In [ ]:
# processor = ChannelMaskingDataset(
#     base_dir="/datax/scratch/jliang/channel_masking_dataset",
#     coarse_channel_width=128,
#     factor=7,
# )

# # just build train/val + YOLO files
# processor.process_scan("my_scan.h5")

# or also dump visualizations & waterfall plots
# processor.process_scan("my_scan.h5", visualize=True)

In [ ]:
# from concurrent.futures import ProcessPoolExecutor, as_completed


# def process_one(h5_path, base_dir, **processor_kwargs):
#     """
#     Wrapper so we can catch exceptions per-file and return status.
#     """
#     try:
#         # initialize per-worker (cheap) 
#         processor = ChannelMaskingDataset(base_dir, **processor_kwargs)
#         processor.process_scan(h5_path, visualize=False)
#         return (h5_path, None)
#     except Exception as e:
#         return (h5_path, e)

# def main(base_dir,
#          num_workers=4,
#          # pass any ChannelMaskingDataset args here:
#          coarse_channel_width=128,
#          factor=5,
#          dilation_size=(10,300),
#          class_id=0,
#          train_frac=0.8,
#          seed=42):

#         # 1) extract your list of H5 paths from the DataFrame
#         #    assume your df is called `df` and the column is "h5_path"
#         h5_list = df[".h5 path"].astype(str).tolist()

#         # 2) prepare shared kwargs for your processor
#         processor_kwargs = {
#             "coarse_channel_width": 128,
#             "factor": 5,
#             "dilation_size": (10,300),
#             "class_id": 0,
#             "train_frac": 0.8,
#             "seed": 42,
#         }

#         # 3) launch parallel processing
#         base_dir = "/datax/scratch/jliang/channel_masking_dataset"
#         num_workers = 8  

#         with ProcessPoolExecutor(max_workers=num_workers) as exe:
#             # submit one task per H5
#             futures = {
#                 exe.submit(process_one, h5, base_dir, **processor_kwargs): h5
#                 for h5 in h5_list
#             }

#             # collect results as they finish
#             for fut in as_completed(futures):
#                 h5_path, error = fut.result()
#                 if error is None:
#                     print(f"{h5_path} done")
#                 else:
#                     print(f"{h5_path} failed:", error)
# if __name__ == "__main__":
#     BASE_DIR = "/datax/scratch/jliang/channel_masking_dataset"
#     main(BASE_DIR, num_workers=8)
